In [68]:
from pathlib import Path
import shutil
import uuid
import json
import hashlib

In [69]:
# Generate the root dir ID
#TODO uncomment the below line after testing is finished
#rootDir = uuid.uuid4()
rootDir = 'testing'

# Get the taskflow ID and Name
taskflowID = '7db37a912d774b58b6412c3d8b2bd782'
taskflowName = 'tf_Generated_Taskflow'


In [70]:
# Create the workspace directories

treePath = Path(f'tmp/{ rootDir }/Explore/Default')
treePath.mkdir(parents=True, exist_ok=True)

In [71]:
# Copy in the generated taskflow XML file
#TODO change to actual generated taskflow file
content = '''<aetgt:getResponse xmlns:aetgt="http://schemas.active-endpoints.com/appmodules/repository/2010/10/avrepository.xsd" xmlns:types1="http://schemas.active-endpoints.com/appmodules/repository/2010/10/avrepository.xsd">
  <types1:Item>
    <types1:EntryId>DEPOiVhC2g0aaaa-gt-58577-2025-02-07T05:28:41.474Z::tf.xml</types1:EntryId>
    <types1:Name>tf_Generated_Taskflow</types1:Name>
    <types1:MimeType>application/xml+taskflow</types1:MimeType>
    <types1:Description/>
    <types1:AppliesTo/>
    <types1:Tags/>
    <types1:VersionLabel>1.0</types1:VersionLabel>
    <types1:State>CURRENT</types1:State>
    <types1:ProcessGroup/>
    <types1:CreatedBy>jbowring_APJS_tspod_SO</types1:CreatedBy>
    <types1:CreationDate>2025-02-07T05:28:41Z</types1:CreationDate>
    <types1:ModifiedBy/>
    <types1:PublicationStatus>unpublished</types1:PublicationStatus>
    <types1:Entry>
      <taskflow xmlns:tfm="http://schemas.active-endpoints.com/appmodules/screenflow/2021/04/taskflowModel.xsd" xmlns:list="urn:activevos:spi:list:functions" xmlns="http://schemas.active-endpoints.com/appmodules/screenflow/2010/10/avosScreenflow.xsd" displayName="tf_Generated_Taskflow" overrideAPIName="false">
        <parameterSet xmlns="http://schemas.active-endpoints.com/appmodules/screenflow/2021/04/taskflowModel.xsd"/>
        <appliesTo/>
        <description/>
        <tags/>
        <generator>Informatica Process Designer 11</generator>
        <notes/>
        <deployment skipIfRunning="false" suspendOnFault="false" tracingLevel="verbose">
          <rest/>
        </deployment>
        <extData>
          <nvpair name="treatEmptyStringAsNotNull">false</nvpair>
          <nvpair name="treatEmptyObjectListAsArray">false</nvpair>
        </extData>
        <flow id="a">
          <start id="b">
            <link id="link1" targetId="c"/>
          </start>
          <end id="c"/>
        </flow>
      </taskflow>
    </types1:Entry>
    <types1:GUID>7db37a912d774b58b6412c3d8b2bd782</types1:GUID>
    <types1:DisplayName>tf_Generated_Taskflow</types1:DisplayName>
  </types1:Item>
  <types1:CurrentServerDateTime>2025-02-07T05:29:29.149Z</types1:CurrentServerDateTime>
</aetgt:getResponse>

'''

with open(f'tmp/{ rootDir }/Explore/Default/{ taskflowName }.TASKFLOW.xml', 'w', encoding='utf-8') as file:
    file.write(content)

In [72]:
# Copy in the Default.Project.json template file
srcPath = Path('templates/Default.Project.json')
tgtPath = Path(f'tmp/{ rootDir }/Explore/Default.Project.json')
shutil.copy(srcPath, tgtPath)



WindowsPath('tmp/testing/Explore/Default.Project.json')

In [73]:
# Generate the ContentsofExportPackage
content = f'''objectPath,objectName,objectType,id
/Explore,Default,Project,92TQndHw9hvjJrQafpLX2R
/Explore/Default,{ taskflowName },TASKFLOW,{ taskflowID }'''

with open(f'tmp/{ rootDir }/ContentsofExportPackage_{ taskflowName }.csv', 'w', encoding='utf-8') as file:
    file.write(content)


In [74]:
# Generate the exportMetadata.v2.json file
with open('templates/exportMetadata.v2.json', 'r', encoding='utf-8') as file:
    exportMetadata = json.load(file)

# Update the taskflow names and ID
exportMetadata['name'] = taskflowName
exportedObjects = []
for obj in exportMetadata['exportedObjects']:
    if obj['objectType'] == 'TASKFLOW':
        obj['objectGuid'] = taskflowID
        obj['objectName'] = taskflowName

    exportedObjects.append(obj)

exportMetadata['exportedObjects'] = exportedObjects

# Save the file
with open(f'tmp/{ rootDir }/exportMetadata.v2.json', 'w', encoding='utf-8') as file:
    json.dump(exportMetadata, file, ensure_ascii=False, indent=4)



In [75]:
# Generate the checksums
with open(f'tmp/{ rootDir }/Explore/Default.Project.json', 'rb') as infile:
        infileBytes = infile.read()
        defaultProjectJsonHash = hashlib.sha256(infileBytes).hexdigest().upper()

with open(f'tmp/{ rootDir }/exportMetadata.v2.json', 'rb') as infile:
    infileBytes = infile.read()
    exportMetadataHash = hashlib.sha256(infileBytes).hexdigest().upper()

with open(f'tmp/{ rootDir }/Explore/Default/{ taskflowName }.TASKFLOW.xml', 'rb') as infile:
    infileBytes = infile.read()
    taskflowHash = hashlib.sha256(infileBytes).hexdigest().upper()

content = f'''#
#Fri Jul 04 02:03:23 UTC 2025
Explore/Default/{ taskflowName }.TASKFLOW.xml={ taskflowHash }
Explore/Default.Project.json={ defaultProjectJsonHash }
exportMetadata.v2.json={ exportMetadataHash }
'''

# Save the checksums file
with open(f'tmp/{ rootDir }/exportPackage.chksum', 'w', encoding='utf-8') as file:
    file.write(content)